In [35]:
import solana
from solana.rpc.api import Client
from solana.publickey import PublicKey
from solana.keypair import Keypair
from solana.transaction import Transaction, TransactionInstruction
from solana.transaction import AccountMeta
from solana.system_program import SYS_PROGRAM_ID
from spl.token import instructions
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
import base58
import base64
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time


network_url = "https://api.devnet.solana.com"
client = Client(network_url)

program_id = PublicKey("BSTpYyZK3X9URjqqkEpERxR14WGLFSGrmbq2Gn7wjyLF")

# Check connection
client.is_connected()


True

In [36]:
seed = bytes([125,215,212,54,213,78,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
client.request_airdrop(payer_keypair.public_key, 2*10**9)
payer_keypair.public_key

tcWyhjGAH8GyH2xBfSJ9tLAKpCkeZ5tRdo3fhAatogp

In [37]:
dsol_wallet_keypair = Keypair.from_secret_key(base58.b58decode("3pz9m7W5ejAEwcQcTeY4695osvh47j1MyEHpMbgScbMtmVY4aicEpwm9HzcHbQCZc3mPJKZUJRYTv5Vpc5iLm9qy"))
print(f"Dsol Pubkey: {dsol_wallet_keypair.public_key}")

(mint_pda, mint_bump) = PublicKey.find_program_address( [b"dsol_mint_pda"], program_id)
print(f"Token Mint ID: {mint_pda}")

dsol_wallet_associated_token_pubkey = instructions.get_associated_token_address(dsol_wallet_keypair.public_key, mint_pda)
print(f"Dsol associated token Pubkey: {dsol_wallet_associated_token_pubkey}")

(dsol_data_pda, dsol_data_bump) = PublicKey.find_program_address( [b"dsol_data_pda", bytes(mint_pda), bytes(dsol_wallet_associated_token_pubkey)], program_id)
print(f"Dsol data Pubkey: {dsol_data_pda}")

(mint_authority_pda, mint_authority_bump) = PublicKey.find_program_address([b"dsol_mint_auth_pda"], program_id)
print(f"Mint Authority Pubkey: {mint_authority_pda}")

associated_token_pubkey = instructions.get_associated_token_address(payer_keypair.public_key, mint_pda)
print(f"Associated Token Pubkey: {associated_token_pubkey}")

payer_account = AccountMeta(payer_keypair.public_key , True, True)
dsol_wallet_account = AccountMeta(dsol_wallet_keypair.public_key , True, True)
dsol_wallet_associated_token_account = AccountMeta(dsol_wallet_associated_token_pubkey , False, True)
dsol_data_pda_account = AccountMeta(dsol_data_pda, False, True)
mint_account = AccountMeta(mint_pda , False, True)
mint_authority_account = AccountMeta(mint_authority_pda , False, False)
associated_token_account = AccountMeta(associated_token_pubkey, False, True)
system_program_account = AccountMeta(SYS_PROGRAM_ID, False, False)
token_program_account = AccountMeta(TOKEN_PROGRAM_ID, False, False)
associated_program_account = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)
rent_account = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
clock_account = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)


Dsol Pubkey: DE2vE88TtWEDNB8Zpqs1YX4rrRkTotwsDXmDdenKAyf5
Token Mint ID: 4hnebtBZBkWF8NvPr4XUdKQFNj2tHim8nqephcMCsBos
Dsol associated token Pubkey: 3bdEvRd2GAVbpfM4XsSYxutzdF7eoYhnnxtiRCiC4B7U
Dsol data Pubkey: 2EsUkyW4zDPfdzHHPqymc3NvvNjEUyFDdrG6tF4ybCaK
Mint Authority Pubkey: 6sg3ZqcNDTc4g3WnbYUDp3b8LE5q7GmkX6Jf2LgAcdag
Associated Token Pubkey: E6VCXz7gmos3nNzqYgJHA84ca9JMoeTUFCyh8caXm3Ra


In [38]:
def create_mint():
    accounts = [
        payer_account,
        dsol_wallet_account,
        dsol_wallet_associated_token_account,
        dsol_data_pda_account,
        mint_account,
        mint_authority_account,
        system_program_account,
        token_program_account,
        associated_token_account,
        rent_account,
        clock_account,

        system_program_account,
        token_program_account,
        associated_program_account,
        token_program_account,
        associated_program_account,
        token_program_account,
        system_program_account,
        system_program_account,
    ]

    transaction = Transaction()
    transaction.add(
        TransactionInstruction(
            accounts,
            program_id,
            bytes([0])
        )
    )

    signature = client.send_transaction(transaction, payer_keypair, dsol_wallet_keypair)['result']
    print(f"https://explorer.solana.com/tx/{signature}?cluster=devnet")


In [39]:
# Create an initial supply of Dsol Tokens
create_mint()

https://explorer.solana.com/tx/3uYA1qW8UsLacJHxS8Emq7NYsGTRNNtwpiorEybeeQeYD6bzyhNLaTcofj2tpvE7pT6LeEGoCmAh6qGjPx1hKjYJ?cluster=devnet


In [40]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')
        
accountSchema = CStruct(
    "date_created" / U32,
    "last_inflation_date" / U32,
)

In [50]:
dsol_data = getAccountInfo(client, dsol_data_pda)
dsol_data

Container(date_created=1645107234, last_inflation_date=1645107434)

In [43]:
def update_mint():
    accounts = [
        payer_account,
        dsol_wallet_associated_token_account,
        mint_account,
        dsol_data_pda_account,
        mint_authority_account,
        token_program_account,
        associated_token_account,
        clock_account,

        token_program_account,
        token_program_account,
        system_program_account,
    ]

    transaction = Transaction()
    transaction.add(
        TransactionInstruction(
            accounts,
            program_id,
            bytes([1])
        )
    )

    signature = client.send_transaction(transaction, payer_keypair, dsol_wallet_keypair)['result']
    print(f"https://explorer.solana.com/tx/{signature}?cluster=devnet")


In [52]:
update_mint()

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: invalid instruction data', 'data': {'accounts': None, 'err': {'InstructionError': [0, 'InvalidInstructionData']}, 'logs': ['Program BSTpYyZK3X9URjqqkEpERxR14WGLFSGrmbq2Gn7wjyLF invoke [1]', 'Program log: Remaining 31535335 secs from now to the next inflation', 'Program BSTpYyZK3X9URjqqkEpERxR14WGLFSGrmbq2Gn7wjyLF consumed 4318 of 200000 compute units', 'Program BSTpYyZK3X9URjqqkEpERxR14WGLFSGrmbq2Gn7wjyLF failed: invalid instruction data']}}

In [ ]:
1645094545 - 1645095145

-600